In [22]:

from sklearn.model_selection import train_test_split
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn import metrics
import pandas as pd
import numpy as np 
from functools import partial

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV

import optuna
from optuna.trial import Trial
optuna.logging.set_verbosity(optuna.logging.FATAL)
import warnings
warnings.filterwarnings("ignore")


In [23]:
features_df = pd.read_csv("bitcoin_train.csv")
features_df.head()

,Unnamed: 0,Date,Open,High,Low,Close,Volume,Daily_Change,Daily_Change_Ind,MACD,...,Open_Gold,Daily_Change_Gold,Daily_Change_Perc_Gold,Increased_Gold,Close/Last_SP500,Open_SP500,Daily_Change_SP500,Daily_Change_Perc_SP500,Increased_SP500,label
0,793,2017-12-28,15864.099609,15888.400391,13937.299805,14606.500000,12336499712,-1232.000000,0.0,636.405515,...,1292.0,5.2,0.004025,1,2687.54,2686.10,1.44,0.000536,1,0.0
1,908,2018-06-14,6342.750000,6707.140137,6334.459961,6675.350098,5138710016,325.450196,1.0,-383.201570,...,1303.1,5.2,0.003990,1,2782.49,2783.21,-0.72,-0.000259,0,0.0
2,224,2015-09-25,234.362000,237.427002,233.684006,235.143997,22363600,0.614990,1.0,-1.731076,...,1151.0,-5.4,-0.004692,0,1931.34,1935.93,-4.59,-0.002371,0,1.0
3,1042,2018-12-26,3819.666748,3893.359619,3769.863770,3857.297607,5326547918,41.806884,1.0,-83.043066,...,1273.5,-0.5,-0.000393,0,2467.70,2363.12,104.58,0.044255,1,1.0
4,563,2017-01-31,920.958984,972.018982,920.958984,970.403015,164582000,50.020996,1.0,12.846431,...,1197.7,13.7,0.011439,1,2278.87,2274.02,4.85,0.002133,1,1.0


In [24]:
features_df.columns

feature_lst = ['Daily_Change', 'Daily_Change_Ind', 'MACD', 'PROC_3', 'PROC_5','PROC_10', 'wpr',\
                'sto_os', 'goog_trend_score', 'count', 'compound', 'retweets_count', 'likes_count', 'replies_count',\
                'compound_weighted_replies', 'compound_weighted_likes','compound_weighted_retweets',\
                'Daily_Change_Perc', 'Daily_Change_Gold', 'Daily_Change_Perc_Gold', 'Increased_Gold', \
                'Daily_Change_SP500', 'Daily_Change_Perc_SP500', 'Increased_SP500']

outcome = features_df.columns[-1]

print(feature_lst)
print(outcome)

['Daily_Change', 'Daily_Change_Ind', 'MACD', 'PROC_3', 'PROC_5', 'PROC_10', 'wpr', 'sto_os', 'goog_trend_score', 'count', 'compound', 'retweets_count', 'likes_count', 'replies_count', 'compound_weighted_replies', 'compound_weighted_likes', 'compound_weighted_retweets', 'Daily_Change_Perc', 'Daily_Change_Gold', 'Daily_Change_Perc_Gold', 'Increased_Gold', 'Daily_Change_SP500', 'Daily_Change_Perc_SP500', 'Increased_SP500']
label


In [25]:
X = features_df[feature_lst]
y = features_df[outcome]
print(X.shape, y.shape)

(1253, 24) (1253,)


In [26]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, random_state = 1234)

-----------------------------------------------
Trying what Ana gave me for hyperparameter tuning (OPTUNA):

In [72]:
# USING OPTUNA TO CHOOSE BEST HYPERPARAMETERS
# this function came from Ana

#https://towardsdatascience.com/state-of-the-art-machine-learning-hyperparameter-optimization-with-optuna-a315d8564de1
def objective(trial:Trial, train = None, labels = None, val = None, val_labels = None, val_rets = None):
    ada_m_depth = trial.suggest_int("max_depth", 1, 5, step = 1)
    ada_n_estimators = trial.suggest_int("n_estimators", 50, 1000, step = 50)
    ada_learning_rate = trial.suggest_float("learning_rate", 0.001, 1.0, step = .009)

    base_estimator = DecisionTreeClassifier(max_depth = ada_m_depth, random_state = 1234)
    ada_clf = AdaBoostClassifier(base_estimator, 
                                n_estimators = ada_n_estimators,
                                learning_rate = ada_learning_rate,
                                algorithm = "SAMME.R")
    ada_clf.fit(X_train, y_train)
    #preds = ada_clf.predict(X_valid)
    accuracy = ada_clf.score(X_valid, y_valid)

    return accuracy

In [73]:
# Still From Ana
#y_valid
# y_valid = np.array(y_valid)
# y_valid = y_valid.reshape(1, -1)
#np.shape(y_valid)
study = optuna.create_study(direction = "maximize")


In [75]:
# Line 3 and 6 is  from ANa

#%%time 

# Line 6 is copy/pasted from Ana, have not updated to our data/var names at this point
# study.optimize(partial(objective,train=train_1,labels=y_1,val=valid,val_labels=y_valid,val_rets=valid_stock_returns), n_trials=20,n_jobs=-1)

# the below code doesn't work at this point but is me trying out oputna.
study.optimize(objective, n_trials = 20, n_jobs = -1)


trial = study.best_trial
print("Best Score: ", trial.value)
print("Best Params: ")
for key, value in trial.params.items():
    print("  {}: {}".format(key, value))

    # create function to pass in objective?

Best Score:  0.7070063694267515
Best Params: 
  max_depth: 1
  n_estimators: 50
  learning_rate: 0.009999999999999998


end of what Ana suggested

-----------------------------------------------------------------------

run a regular decision tree model with different tree depth, 
use as comparison to show that adaboost is better

In [ ]:
# Figure out what the tree depth should be (maximum score)
results_dict = {}
cx_validation = KFold(n_splits = 10, shuffle = True, random_state = 1)
for depth in range(1, 11):
    base_estimator = DecisionTreeClassifier(max_depth = depth, random_state = 1234)
    if base_estimator.fit(X_train,y_train).tree_.max_depth < depth:
        break
    accuracy_score = np.mean(cross_val_score(base_estimator, X, y, cv = cx_validation, scoring = 'accuracy', n_jobs = 1))
    print("Depth:", depth, ", Score:", accuracy_score)

    results_dict[depth] = accuracy_score

max_val = -1
max_depth_val = None
for key, val in results_dict.items():
    if val > max_val:
        max_val = val
        max_depth_val = key

print("Max depth should be:", max_depth_val)

    # https://educationalresearchtechniques.com/2019/01/02/adaboost-classification-in-python/
    # https://towardsdatascience.com/boosting-and-adaboost-clearly-explained-856e21152d3e
    # https://python-bloggers.com/2019/01/adaboost-classification-in-python/ # this one for 
    

Depth: 1 , Score: 0.6327555555555555
Depth: 2 , Score: 0.6455619047619047
Depth: 3 , Score: 0.6512507936507935
Depth: 4 , Score: 0.646431746031746
Depth: 5 , Score: 0.6440253968253967
Depth: 6 , Score: 0.6313523809523809
Depth: 7 , Score: 0.6184888888888889
Depth: 8 , Score: 0.6025269841269841
Depth: 9 , Score: 0.5969523809523809
Depth: 10 , Score: 0.6049015873015873
Max depth should be: 3


Straight run of the model with default chosen n_estimators and learning_rate

In [63]:
# Create adaboost classifer object
base_estimator = DecisionTreeClassifier(max_depth = 1, random_state = 1234)

adaboost = AdaBoostClassifier(base_estimator = base_estimator, random_state = 1234)

# Train Adaboost Classifer
adaboost.fit(X_train, y_train)

# Predict the response for valid dataset
y_hat = adaboost.predict(X_valid)

In [64]:
# Model Accuracy with chosen decision tree depth and randomly chosen hyperparameters
accuracy_score = metrics.accuracy_score(y_valid, y_hat)
mse = metrics.mean_squared_error(y_valid, y_hat)
confusion = metrics.confusion_matrix(y_valid, y_hat) 

print("Accuracy:", accuracy_score)
print("MSE:", mse)
print("CONFUSION:", confusion)

## Consider changing to this wiht the cross val score even for default/trying things out.
# Probably leave at decision stump instead of max depth thing
# https://medium.com/@chaudhurysrijani/tuning-of-adaboost-with-computational-complexity-8727d01a9d20

Accuracy: 0.7070063694267515
MSE: 0.2929936305732484
CONFUSION: [[ 91  45]
 [ 47 131]]


In [67]:
feature_import = adaboost.feature_importances_
for i in range(len(feature_lst)):
    print(f"{feature_lst[i]}: {feature_import[i]}")

Daily_Change: 0.04
Daily_Change_Ind: 0.0
MACD: 0.04
PROC_3: 0.1
PROC_5: 0.06
PROC_10: 0.08
wpr: 0.06
sto_os: 0.1
goog_trend_score: 0.04
count: 0.02
compound: 0.02
retweets_count: 0.06
likes_count: 0.0
replies_count: 0.04
compound_weighted_replies: 0.0
compound_weighted_likes: 0.08
compound_weighted_retweets: 0.04
Daily_Change_Perc: 0.12
Daily_Change_Gold: 0.02
Daily_Change_Perc_Gold: 0.04
Increased_Gold: 0.0
Daily_Change_SP500: 0.02
Daily_Change_Perc_SP500: 0.02
Increased_SP500: 0.0


hyper parametertuning (number estimators and learning rate)


tuning n_estimators and learning_Rate

In [ ]:
# https://machinelearningmastery.com/adaboost-ensemble-in-python/

# alternative way to check n_estiamtors and learning_Rate
model = AdaBoostClassifier()
# define the grid of values to search
grid = dict()
grid['n_estimators'] = [10, 50, 100, 500, 1000]
grid['learning_rate'] = [0.001, 0.01, 0.1, 1.0]

# define the evaluation procedure
cv = RepeatedStratifiedKFold(n_splits = 10, n_repeats = 3, random_state = 1)

# define the grid search procedure
grid_search = GridSearchCV(estimator = model, param_grid = grid, n_jobs = -1, cv = cv, scoring = 'accuracy')

# execute the grid search
grid_result = grid_search.fit(X_train, y_train)

# summarize the best score and configuration
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

# summarize all scores that were evaluated
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Rerun adaboost with chosen hyperparameters

In [68]:
base_estimator = DecisionTreeClassifier(max_depth = 1, random_state = 1234)

chosen_learning_rate = list(grid_result.best_params_.values())[0]
chosen_n = list(grid_result.best_params_.values())[1]

print("chosen_n:", chosen_n)
print("chosen_learing_rate:", chosen_learning_rate)

adaboost = AdaBoostClassifier(base_estimator = base_estimator, n_estimators = chosen_n,
                              learning_rate = chosen_learning_rate, random_state = 1234)

# Train Adaboost Classifer
adaboost.fit(X_train, y_train)

# Predict the response for valid dataset
y_hat = adaboost.predict(X_valid)

chosen_n: 50
chosen_learing_rate: 0.1


In [70]:
# Model Accuracy with chosen decision tree depth and chosen hyperparameters
accuracy_score = metrics.accuracy_score(y_valid, y_hat)
mse = metrics.mean_squared_error(y_valid, y_hat)
confusion = metrics.confusion_matrix(y_valid, y_hat) 

print("Accuracy:", accuracy_score)
print("MSE:", mse)
print("CONFUSION:", confusion)

Accuracy: 0.7165605095541401
MSE: 0.28343949044585987
CONFUSION: [[ 90  46]
 [ 43 135]]


In [71]:
feature_import = adaboost.feature_importances_
for i in range(len(feature_lst)):
    print(f"{feature_lst[i]}: {feature_import[i]}")

Daily_Change: 0.08
Daily_Change_Ind: 0.0
MACD: 0.0
PROC_3: 0.24
PROC_5: 0.26
PROC_10: 0.0
wpr: 0.14
sto_os: 0.08
goog_trend_score: 0.0
count: 0.0
compound: 0.0
retweets_count: 0.0
likes_count: 0.0
replies_count: 0.0
compound_weighted_replies: 0.0
compound_weighted_likes: 0.0
compound_weighted_retweets: 0.0
Daily_Change_Perc: 0.18
Daily_Change_Gold: 0.0
Daily_Change_Perc_Gold: 0.02
Increased_Gold: 0.0
Daily_Change_SP500: 0.0
Daily_Change_Perc_SP500: 0.0
Increased_SP500: 0.0


In [21]:
# now try adjusting depth? This needs to be done simultaneously if at all...might be where optuna is helpful
score = []
for depth in [1,2,3] : 
    base_estimator = DecisionTreeClassifier(max_depth = depth, random_state = 1234)
    adaboost = AdaBoostClassifier(base_estimator = base_estimator, n_estimators = chosen_n,
                              learning_rate = chosen_learning_rate, random_state = 1234)
    scores_ada = cross_val_score(adaboost, X_train, y_train)
    score.append(scores_ada.mean())

In [20]:
print("score", score)

score [0.6581400729489901, 0.6720017420654364, 0.6113560890630954]


In [ ]:
''' other links
https://inria.github.io/scikit-learn-mooc/python_scripts/ensemble_adaboost.html
https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostClassifier.html
 https://scikit-learn.org/stable/tutorial/statistical_inference/model_selection.html
 https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html
 
 https://towardsdatascience.com/how-do-you-implement-adaboost-with-python-a76427b0fa7a
 https://towardsdatascience.com/machine-learning-part-17-boosting-algorithms-adaboost-in-python-d00faac6c464
 
 https://machinelearningmastery.com/adaboost-ensemble-in-python/
 https://machinelearningmastery.com/k-fold-cross-validation/

 https://medium.datadriveninvestor.com/understanding-adaboost-and-scikit-learns-algorithm-c8d8af5ace10
 https://blog.paperspace.com/adaboost-optimizer/
 https://www.datacamp.com/tutorial/adaboost-classifier-python
 https://educationalresearchtechniques.com/2019/01/02/adaboost-classification-in-python/

 consider trying standardization https://github.com/mehuls45/Heart-Disease-prediction-using-ML/blob/master/AdaBoost.ipynb

 # no need to normalize/standardize in ensemble methods 
 https://towardsdatascience.com/the-ultimate-guide-to-adaboost-random-forests-and-xgboost-7f9327061c4f

 https://github.com/julianikulski/bike-sharing/blob/master/bike_sharing_demand.ipynb

https://www.google.com/search?client=firefox-b-1-d&q=hyperparameter+tuning+adaboost
 https://stackoverflow.com/questions/32210569/using-gridsearchcv-with-adaboost-and-decisiontreeclassifier

# Repeat for Dogecoin


In [ ]:
features_df = pd.read_csv("bitcoin_train.csv")
features_df.head()

--------------------------------------------------------------------
MISC OTHER CODE NOT USING

-------------------------------------------------------------
We will evaluate the model using repeated stratified k-fold cross-validation, with three repeats and 10 folds. We will report the mean and standard deviation of the accuracy of the model across all repeats and folds.

https://machinelearningmastery.com/adaboost-ensemble-in-python/

In [45]:
model = AdaBoostClassifier()
# evaluate the model
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1234)
n_scores = cross_val_score(model, X_train, y_train, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')
# report performance
print('Accuracy: %.3f (%.3f)' % (np.mean(n_scores), np.std(n_scores)))

Accuracy: 0.635 (0.053)


This achieved 63.5% accuracy with default hyperparameters. Now make predictions on new data

In [47]:
model = AdaBoostClassifier()
# fit the model on the whole dataset
model.fit(X_train, y_train)
# make a single prediction
y_hat = model.predict(X_valid)

accuracy_score = metrics.accuracy_score(y_valid, y_hat)
mse = metrics.mean_squared_error(y_valid, y_hat)
confusion = metrics.confusion_matrix(y_valid, y_hat) 

print("Accuracy:", accuracy_score)
print("MSE:", mse)
print("CONFUSION:", confusion)

Accuracy: 0.7070063694267515
MSE: 0.2929936305732484
CONFUSION: [[ 91  45]
 [ 47 131]]


In [53]:
base_estimator = DecisionTreeClassifier(max_depth=3, random_state=1234)
adaboost = AdaBoostClassifier(base_estimator=base_estimator,
                              n_estimators=3, algorithm="SAMME",
                              random_state=0)
adaboost.fit(X_train, y_train)

AdaBoostClassifier(algorithm='SAMME',
                   base_estimator=DecisionTreeClassifier(max_depth=3,
                                                         random_state=1234),
                   n_estimators=3, random_state=0)

In [58]:
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
from sklearn.inspection import DecisionBoundaryDisplay


ImportError: cannot import name 'DecisionBoundaryDisplay' from 'sklearn.inspection' (/home/ksarussi/.local/lib/python3.8/site-packages/sklearn/inspection/__init__.py)

In [56]:
for boosting_round, tree in enumerate(adaboost.estimators_):
    plt.figure()
    # we convert `data` into a NumPy array to avoid a warning raised in scikit-learn
    DecisionBoundaryDisplay.from_estimator(
        tree, X_train.to_numpy(), response_method="predict", cmap="RdBu", alpha=0.5
    )
    sns.scatterplot(x=culmen_columns[0], y=culmen_columns[1],
                    hue=target_column, data=penguins,
                    palette=palette)
    plt.legend(bbox_to_anchor=(1.04, 0.5), loc="center left")
    _ = plt.title(f"Decision tree trained at round {boosting_round}")

NameError: name 'DecisionBoundaryDisplay' is not defined

<Figure size 432x288 with 0 Axes>

In [ ]:
# hyper parametertuning (number estimators and learning rate)
# ada = AdaBoostClassifier()
# search_grid = {'n_estimators':[500,1000,2000], 'learning_rate':[.001,0.01,.1]}
# search = GridSearchCV(estimator = ada, param_grid = search_grid, scoring = 'accuracy', n_jobs = 1, cv = cx_validation)

#https://python-bloggers.com/2019/01/adaboost-classification-in-python/

# search.fit(X,y)
# search.best_params_
# print(search.best_params_)
# search.best_score_
# print(search.best_score_)
#Out[34]: 0.7425149700598802



# https://python-bloggers.com/2019/01/adaboost-classification-in-python/

# score = np.mean(cross_val_score(ada, X, y, scoring='accuracy', cv = cx_validation, n_jobs = 1))
# score


#https://python-bloggers.com/2019/01/adaboost-classification-in-python/

# AdaBoostClassifier.get_params().keys()